# Demand densities application

## WFS custom function for downloading vector layers from geoserver to the disk through OWSLib.

In [ ]:
import os
import pandas as pd
from owslib.wfs import WebFeatureService
import json

wfs = WebFeatureService(url='http://127.0.0.1:8082/geoserver/wfs', version='1.1.0')
def getFeature(layer, bbox=None, workspace="geonode", format="json"):
    if bbox:
        response = wfs.getfeature(typename=workspace+":"+layer, outputFormat=format, bbox=bbox)
    else:
        response = wfs.getfeature(typename=workspace+":"+layer, outputFormat=format)
    with open(os.path.join("wfs", layer+"."+format), 'wb') as f:
        while True:
            data = response.read(100)
            if not data:
                break
            f.write(data)
    name=f.name
    f.close
    return (name)

## WFS requests to geoserver in order to store layer as vector (csv & GeoJson) to the disk

In [ ]:
import folium
from folium.plugins import HeatMap

layer = "kwh_sq"
geodata = getFeature(layer, format="json")
data = pd.read_csv(getFeature(layer, format="csv"))
data.head()
with open(os.path.join("wfs",layer+".json")) as f:
    data_geojson = json.loads(f.read())

## I Also import dataset from postgis to get geometry columns in geopandas

In [ ]:
## I also import the data from postgres
import psycopg2
import geopandas as gpd
conn = psycopg2.connect(database="mydb",
  user="user",
  host="localhost",
  password="user")
kwh_sq = gpd.read_postgis(
     'select gid, mun, heat, water, geom, St_AsText(St_Centroid(geom)) as center from final', 
    conn, geom_col='geom')

## Here I create the data as required for the heatmaps

I create centroids from polygons and then link them to the demand density metrics

In [ ]:
import numpy as np
poly_to_x = lambda k:k.centroid.x
poly_to_y = lambda k:k.centroid.y
centroids_x = kwh_sq.geom.apply(poly_to_x).values
centroids_y = kwh_sq.geom.apply(poly_to_y).values
heat = kwh_sq.heat.values
water = kwh_sq.water.values
heat_data = np.vstack((centroids_y, centroids_x, heat)).T
water_data = np.vstack((centroids_y, centroids_x, water)).T

## Map

In [ ]:
m = folium.Map([37, 22], tiles='cartodbdark_matter', zoom_start=6)

# HeatMap layers
HeatMap(data=heat_data,
        name="Heat demand density",
        radius=12).add_to(m)

HeatMap(data=water_data,
        name="Water demand density",
        radius=13,
        show=False).add_to(m)

# Main vector layer
folium.GeoJson(
    #data = kwh_sq[['mun', 'geom', 'heat', 'water']].to_json(),
    data=geodata,
    name='Density',
    show=True,
    style_function=lambda x: {
        'fillColor': 'lightblue',
        'color': 'black',
        'opacity': 0.3,
        'weight': 1,
        'fillOpacity':0.3
    },
    highlight_function=lambda x: {
        'fillOpacity':0.9
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['mun', 'heat', 'water'],
        aliases=['Municipality:', 'Energy demand for heat [kwh/sq_m]:', 'Energy demand for water [kwh/sq_m]:']
    )
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
m.save(os.path.join('results', 'demand_densities.html'))